# Recursive CTEs


Today, we're going to introduce recursion and recursive CTEs, which is probably the most difficult aspect in our entire course. 

Don't worry if you can't understand something at first – take your time and read the instructions carefully again and again.

Recursive CTEs make it possible to process 'hierarchical structures' – trees and graphs – using SQL. 

For instance, such structures are used when talking about superior and inferior employees, railway connections, application menus, etc.

What we're going to show here is standard syntax, but you may expect some deviations in various databases. 

We're using Postgres, and the same syntax should work in SQL Server. 

Oracle supports recursive CTEs since version 11gR2. 

MySQL will support them from version 8.0. 

Some other databases use an alternative syntax with CONNECT BY, but we won't cover it here.

For right now, we're gonna play around with a blank fiddle. Go here: https://www.db-fiddle.com/

# Recursion in the real world

Let's get started, then. 

You might, or might not, have heard about recursion before. 

In either case, it's good to clarify what it is first.

Is there recursion in real life? 

Yes, there is. 

Take a good look at the Romanesco broccoli shown below. 

Its cones are made of smaller cones which are, in turn, made of yet smaller cones that are made of even smaller cones...

<img src='https://www.summerhillmarket.com/wp-content/uploads/2021/05/romanesco-broccoli-one-head.png'>

Another example could be the famous Russian matryoshka dolls. 

If you open the biggest doll, you will see a smaller one. 

Once the smaller one is opened, you can see yet a smaller one, which in turn has another smaller doll inside itself.

<img src = 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Matryoshka_transparent.png/220px-Matryoshka_transparent.png'>

In each case, we can say that a pattern repeats itself. 

Formally speaking, recursion takes place when something is defined in terms of itself. 

A Romanesco broccoli cone is a cone made of smaller Romanesco broccoli cones. 

A matryoshka doll is a doll which holds a smaller matryoshka doll inside.

# Recursion in Computer Science 

In the world of computer science, recursion takes place when we invoke a block of instructions inside itself.

In SQL, we talk about a recursive CTE when that CTE refers to itself. 

The idea behind recursive CTEs is usually as follows: 'Hey, give me everything you have so far, but add something extra as well. Then, repeat this procedure until I tell you to stop. By adding small pieces in each recursive step, I expect to finally arrive at the right result.'

Sounds vague? 

Let's introduce our first example.

### Exercise

Let's say we want to show numbers from 1 to 10, but we don't have any table with rows containing those numbers.

Luckily, we can use recursion. 

Take a look at the template, run it and see the result. 

Don't pay too much attention to the details – we'll explain everything in a second.

```
WITH RECURSIVE counter(previous_value) AS (
  SELECT 1
  UNION ALL 
  SELECT counter.previous_value + 1
  FROM counter
  WHERE previous_value < 10
)

SELECT *
FROM counter;
```

# Recursion Explained

As you could see, we got rows containing numbers from 1 to 10 without actually using any table. 

Now, it's time to explain the magic behind that query. 

Take another look at the template from the previous exercise:

<img src='https://learnsql.com/static/common-table-expressions-Part4_diagrams1.png'>

In Postgresql, a recursive CTE starts with the words WITH RECURSIVE, followed by the CTE name and the list of columns in the parentheses. 

In this case, our CTE under the name counter has only one column, which is the previous_value (that is, the value shown in the previous step).

Each correctly constructed recursive CTE has three essential elements inside the parentheses: an anchor member, a recursive member and a terminator. 

Let's discuss each of them.

An `anchor member` is basically something (an anchor point) that we start with.

As we want to show numbers from 1 to 10, we will start with the number 1. 

You could think of it as the smallest matryoshka doll which is so small that it has no other matryoshka doll inside. 

In our case, the anchor member is expressed as

```
SELECT 1
```

Then comes the `recursive member`. 

The recursive member tells us what to do in each recursive step. 

In the case of matryoshka dolls, this could be: create a bigger matryoshka doll which will have the previous matryoshka doll inside it. 

So there we go with numbers: given that you have all rows with numbers generated so far, add another row with a number that is bigger by 1 than the number in the row previously generated. 

In our case

```
SELECT previous_value + 1
FROM counter
```

At this point, the CTE counter refers to itself (FROM counter).

If we stopped at this point, our query would go on and on, and it would never stop. 

In our real-life examples, each recursion comes to an end: there is the biggest matryoshka doll that holds all smaller dolls inside itself, just as there is the biggest cone in a Romanesco broccoli.

We need a terminator in recursive CTEs as well: something that will eventually stop our recursion. 

In this case, it's

```
WHERE previous_value < 10
```

This means: as long as the previous value is smaller than 10, continue with the recursion. 

But if you reach 10, don't add any new rows. 

When a recursive CTE has nothing new to add in a given step, it terminates.

Now, let's discuss the remaining parts of our example query. I'll show it again here again for your convenience:

<img src='https://learnsql.com/static/common-table-expressions-Part4_diagrams1.png'>

First of all, note that we used `UNION ALL` to join the anchor member (`SELECT 1`) with all further recursive steps (`SELECT previous_value + 1`). 

Instead of `UNION ALL`, we could also use `UNION`, which will remove duplicate values from the recursion. 

In our case, no duplicate values exist, so there is no difference between `UNION` and `UNION ALL`.

Once the recursive CTE is completed and our temporary table is created, we need to query this table to actually see any results. 

That's why we put

```
SELECT *
FROM counter
````

outside the CTE. This part of the query is called `invocation`.

The animation below shows how our final recursive table is constructed.

<img src='https://learnsql.com/static/common-table-expressions-Part4_diagrams2.png'>

Technically speaking, recursive CTEs do not use recursion, but iterations. 

Still, as the SQL keyword for this kind of queries is RECURSIVE, it is common to call such queries recursion. 

Treat it as some sort of convention rather than a technically accurate description.

### Exercise

Modify the example query so that it shows numbers from 20 to 5 (inclusive, in descending order).


```
WITH RECURSIVE counter (previous_value) AS (
  SELECT 20
  UNION ALL
  SELECT previous_value - 1
  FROM counter
  WHERE previous_value > 5
)
SELECT * 
FROM counter;
```


# Recursion explained: number of columns

Of course, you can have more than one column in your recursive CTE. 

Do remember, though, that the `anchor member` and the `recursive member` must have the same number of columns – that's an essential condition for any query with `UNION` or `UNION ALL`.

Take a look:

```
WITH RECURSIVE counter (
  previous_value, previous_double) AS (

  SELECT 1, 2  /*anchor member*/

  UNION ALL

  /*recursive member*/
  SELECT
    previous_value + 1, 
    2 * (previous_value + 1)
  FROM counter

  /*termination check*/
  WHERE previous_value < 10
)

/*invocation*/
SELECT
  *
FROM counter;
```

In the query above, we show numbers from 1 to 10 plus the respective number multiplied by two.

In the recursive member, we added `2 * (previous_value + 1)` as the second column. 

Because the number of columns must be the same in the anchor and recursive member, we had to add the second column in the anchor member as well (which is just the number 2, that is 1 multiplied by 2).

Naturally, we also added the second column to the list of CTE columns:

```
WITH RECURSIVE counter (
  previous_value, previous_double) AS ...
```

### Exercise

Show two columns. In the first column (column name previous_value), show numbers from one to ten. In the second column (column name previous_sum), show the total sum of all numbers so far.

```
WITH RECURSIVE counter (previous_value, previous_sum) AS (
  SELECT 1, 1 
  UNION ALL
  SELECT previous_value + 1,previous_sum + previous_value + 1 
  FROM counter
  WHERE previous_value < 10
)
SELECT *
FROM counter;
```


# Exercise

Create a recursive query that will show three columns:

- `num` – even numbers from 2 to 20,
- `num_plus_prev` – that number plus the previous number,
- `num_square` – the number squared.

```
WITH RECURSIVE maths (num, num_plus_prev, num_square) AS (
  SELECT 2, 2, 4
  UNION ALL
  SELECT num + 2, num + num + 2, (num + 2) * (num + 2)
  FROM maths
  WHERE num < 20
)
SELECT *
FROM maths;
```

# Multiple CTEs

Just as you can have multiple columns in your CTE, you can also have multiple CTEs, some of which may be recursive, others not. 

The rule for syntax is as follows: if at least one CTE is recursive, start the whole query with `WITH RECURSIVE`, and then define your CTEs just as you would normally do. 


Take a look at the example below.

```
WITH RECURSIVE prime(x) AS (
  SELECT    2
  UNION 
  SELECT    3
  UNION
  SELECT    5
  UNION
  SELECT    7
  UNION
  SELECT   11
),

total(multiply,prime,score) AS (
  SELECT 
    0,
    0,
    0
  UNION 
  SELECT 
    multiply+1,
    x, 
    x * (multiply+1)
  FROM prime, total 
  WHERE multiply < 3
)

SELECT 
  * 
FROM total
ORDER BY multiply, prime;
```

In the example, we first create a list of the first 5 prime numbers, and then we create a recursive CTE that calculates multiples of these prime numbers. 

Note that the word RECURSIVE is always used before the first CTE, even if that query is not recursive.


### Exercise

Look at the template. 

There is already a single CTE named letters defined for you. 

Your task is to complete the words CTE so that it generates all possible words made from those letters that have at most 5 letters. 

The shortest word should be an empty word ('').

```
WITH RECURSIVE letters(x) AS (
  SELECT 'a' UNION SELECT 'b'
),
words(w) AS (
  -- your code goes here
  SELECT ''
  UNION
  SELECT x || w
  FROM words, letters
  WHERE length(w) < 5
)
SELECT *
FROM words;
```

# Mind Infinity

With this kind of looping queries, it's important to always have a termination check that will end the query at some point. 

Let's see what happens if we forget about that.

```
WITH RECURSIVE counter(previous_value) AS (
  SELECT
    1 -- anchor member
  UNION ALL
  SELECT 
    previous_value + 1 -- recursive member
  FROM counter -- termination check
)
SELECT *
FROM counter; -- invocation
```

Look at the template. It's essentially the first example query from the previous section, but we removed the termination check. 

Try to run the query and find out for yourself what happens when the recursion never stops

# LIMIT keyword

As you could see, our query terminated with an error, but it's only because of the special database settings in our course. 

Normally, the database would not stop the query on its own.

As a result, it would freeze and you would have to break the query manually. 

Please remember about that!

In Postgres, you can use another keyword `LIMIT n`, where n is the maximal number of rows you expect. 

Even if your query does not contain a termination check (or it does, but the check is faulty), LIMIT will take care of query termination:

```
WITH RECURSIVE counter(previous_value) AS (
  SELECT
    1 -- anchor member
  UNION ALL
  SELECT
    previous_value + 1 -- recursive member
  FROM counter
)

SELECT 
  * 
FROM counter -- invocation
LIMIT 10; -- termination check
```

# Trees in computer science

Now that you got the basics, we will learn some real-world examples.

Let's say we have a company with a hierarchical structure, i.e. each employee has a superior (except for the boss). 

An employee has precisely one superior, but one superior may have multiple inferiors. 

An example structure may look as follows:

<img src='https://learnsql.com/static/common-table-expressions-Part4_diagrams3.png'>

In computer science, we call such a structure a tree, where each node (employee) has a parent node (superior), except for the root (boss), which has no parent node.

How can we store these hierarchical relations in a database? 

It's quite simple: each employee is represented with a row that has a column superior_id with the ID of their superior.

<img src='https://learnsql.com/static/common-table-expressions-Part4_diagrams6.png'>

Let's move to a fiddle now:https://www.db-fiddle.com/f/a6DHkd8V71VMKLKjZthEh2/0


### Exercise

Select all data from table employee.

As you can see, the table has the following columns:

- `id` – idetifier of employee,
- `first_name` – first name of employee,
- `last_name` – last name of employee,
- `superior_id` – identifier of person above employee.

### Exercise

The template query you see will show each employee with their path to the boss. 

Run the query and see the results. 

For now, don't worry about the details.

```
WITH RECURSIVE hierarchy AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id,
    'Boss' AS path
  FROM employee
  WHERE superior_id IS NULL
  UNION ALL 
  SELECT
    employee.id,
    employee.first_name,
    employee.last_name,
    employee.superior_id,
    hierarchy.path || '->' || employee.last_name
  FROM employee, hierarchy
  WHERE employee.superior_id = hierarchy.id
)

SELECT *
FROM hierarchy;
```

I'll now help you construct the query step-by-step so that you understand how it works.

In the first step, we'll show information about the boss. 

It is the only person in the table that has a NULL superior_id. 

For that person, show the id, first_name, last_name, superior_id, and the last column called path with 'Boss' inside.

```
SELECT
  id,
  first_name,
  last_name,
  superior_id,
  'Boss' AS path
FROM employee
WHERE superior_id IS NULL;
```

The result of the above query is already in the template. 

Your task is to show not only the boss, but also his immediate subordinates. 

For them, the path column should contain 'Boss->' plus the last_name of the respective employee. 

Take the results from the template and UNION them with the boss's immediate subordinates.

```
WITH boss AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id,
    'Boss' AS path
  FROM employee
  WHERE superior_id IS NULL
)

SELECT 
  id, 
  first_name, 
  last_name, 
  superior_id, 
  'Boss' AS path 
FROM boss 
UNION
SELECT
  employee.id,
  employee.first_name,
  employee.last_name,
  employee.superior_id,
  'Boss' || '->' || employee.last_name 
FROM employee,boss 
WHERE employee.superior_id = boss.id;
```

Now comes the hardest part. 

You are given the query from the previous exercise, but you need to transform it so that it goes deep down to every employee in the path. 

As a result, we want to see each employee in the company with their data and the path from Boss to that person.

```
WITH RECURSIVE hierarchy AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id,
    'Boss' AS path
  FROM employee
  WHERE superior_id IS NULL
  UNION ALL 
  SELECT
    employee.id,
    employee.first_name,
    employee.last_name,
    employee.superior_id,
    hierarchy.path || '->' || employee.last_name
  FROM employee, hierarchy
  WHERE employee.superior_id = hierarchy.id
)
SELECT *
FROM hierarchy;
```

# Common error with recursive CTEs

We'll now see a very common mistake with recursive CTEs. 

What is the error? 

Let's see it in an exercise.

# Exercise 

We have changed the value 'Boss' in the anchor query to column last_name.

Try to run the query. 

What do you think will happen?

```
WITH RECURSIVE hierarchy AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id,
    last_name AS path
  FROM employee
  WHERE superior_id IS NULL
  UNION ALL 
  SELECT
    employee.id,
    employee.first_name,
    employee.last_name,
    employee.superior_id,
    hierarchy.path || '->' || employee.last_name
  FROM employee, hierarchy
  WHERE employee.superior_id = hierarchy.id
)

SELECT *
FROM hierarchy;
```

# Use CAST with recursive CTEs

Oops, our query failed. 

That was unexpected. 

What happened?

If you carefully read the error message. It will tell you that the types in non-recursive and recursive parts of our query do not match.

The data type for

```
SELECT
  ...
  last_name as path
```

is the data type for column last_name, i.e. varchar.

The data type for

```
SELECT
  ...
  hierarchy.path || '->' || employee.last_name
```

is text. 

Text and varchar types do not match. 

The `UNION ALL` operator expects the types for corresponding columns to match, so the query fails.

What can we do about it?

We have to explicitly cast column last_name to match the type of the other expression, like this:

```
SELECT
  ...
  CAST(last_name AS text) AS path
```

Unfortunately, there are no simple rules explaining to which type you have to cast in order to get your query right. 

This depends on the database engine you use and its rules for handling data types.

# Exercise

Fix the query so that it runs correctly.

Cast the column last_name to type text.

```
WITH RECURSIVE hierarchy AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id,
    CAST(last_name AS text) AS path
  FROM employee
  WHERE superior_id IS NULL
  UNION ALL 
  SELECT
    employee.id,
    employee.first_name,
    employee.last_name,
    employee.superior_id,
    hierarchy.path || '->' || employee.last_name
  FROM employee, hierarchy
  WHERE employee.superior_id = hierarchy.id
)

SELECT *
FROM hierarchy;
```




# Homework

https://www.db-fiddle.com/f/a6DHkd8V71VMKLKjZthEh2/0

### Exercise

Let's assume that 'distance' in our hierarchical company is the number of people from the Boss to that person. 

According to this definition, Boss will have a distance of 0, her direct subordinates: 1, their subordinates: 2 etc.

Now, add another column (name distance) to the template query which shows the distance for each employee.

```
WITH RECURSIVE hierarchy AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id,
    'Boss' AS path,
    0 AS distance
  FROM employee
  WHERE superior_id IS NULL
  UNION ALL 
  SELECT
    employee.id,
    employee.first_name,
    employee.last_name,
    employee.superior_id,
    hierarchy.path || '->' || employee.last_name,
    hierarchy.distance + 1
 FROM employee, hierarchy
 WHERE employee.superior_id = hierarchy.id
)
SELECT *
FROM hierarchy;
```



# Exercise

Show the id, first, last name, superior_id of each subordinate of the employee with id = 2 (not only the direct subordinates, but also their subordinates etc.) and the employee herself.

```
WITH RECURSIVE hierarchy AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id
  FROM employee
  WHERE id = 2
  UNION ALL 
  SELECT
    employee.id,
    employee.first_name,
    employee.last_name,
    employee.superior_id
  FROM employee, hierarchy
  WHERE employee.superior_id = hierarchy.id
)

SELECT *
FROM hierarchy;
```

# Exercise

Show only the path from the Boss to the employee with id = 10.



```
WITH RECURSIVE hierarchy AS (
  SELECT
    id,
    first_name,
    last_name,
    superior_id,
    'Boss' AS path
  FROM employee
  WHERE superior_id IS NULL
  UNION ALL 
  SELECT
    employee.id,
    employee.first_name,
    employee.last_name,
    employee.superior_id,
    hierarchy.path || '->' || employee.last_name
  FROM employee, hierarchy
  WHERE employee.superior_id = hierarchy.id
)
SELECT
  path
FROM hierarchy
WHERE id = 10;
```